<a href="https://colab.research.google.com/github/Romal27/DSGP-24-Retina94/blob/component-1/Pretrained.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import cv2
import numpy as np
from skimage.feature import hog
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.models import Model



cv2 - library used for image processing (OpenCV)
numpy - numerical computation
hog - histogram of oriented gradient - for texture and shape detection
VGG16 and preprocess_input - pre trained model and its preprocessing function for extracting features
Model - Keras object to create a feature extraction model from VGG16

In [ ]:
from google.colab import files

uploaded = files.upload()
image_paths = list(uploaded.keys())


Saving orange circle.jpg to orange circle.jpg


In [ ]:
def preprocess_image(image_path, target_size=(224, 224)):

    image = cv2.imread(image_path)

    image_resized = cv2.resize(image, target_size)

    image_normalized = image_resized / 255.0
    return image_normalized


Load the image from path
224x224 is the required size for VGG16
Pixel value is normalize to fit in with the neural network
(Here this preprocessing is done for the image to fit in with the requirement of VGG16)

In [ ]:
def extract_hog_features(image_path):


    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)  # Load image as grayscale
    if image is None:
        raise ValueError(f"Could not read the image from {image_path}")


    image_resized = cv2.resize(image, (128, 128))  # Resize to 128x128 pixels

    # Extract HOG features
    features, hog_image = hog(
        image_resized,
        orientations=9, # Divide the gradient extraction into 9 bins
        pixels_per_cell=(8, 8),# DIvide the image into cells 8x8 pixels
        cells_per_block=(2, 2), # Block 2x2 cells for feature extraction
        visualize=True,  # Set to True for visualization
    )
    return features



HOG focus on texture, NOT color.
Turns the image into greyscale.
HOG captures edges and shapes and texture of fundus images

In [ ]:
def detect_circles(image_path):

    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    image_blurred = cv2.medianBlur(image, 5)
    circles = cv2.HoughCircles(image_blurred, cv2.HOUGH_GRADIENT, dp=1.2, minDist=30,
                               param1=50, param2=30, minRadius=20, maxRadius=50)
    return circles


Converts to a greyscale
Apply meadium blur for noise reduction
Using Hough Circle Transform to detect the circular features

In [ ]:
def extract_vgg16_features(image_path, target_size=(224, 224)):

    # Load the pre-trained VGG16 model
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    feature_extractor = Model(inputs=base_model.input, outputs=base_model.layers[-1].output)
    image = preprocess_image(image_path, target_size)
    image = np.expand_dims(image, axis=0)
    image = preprocess_input(image)

    features = feature_extractor.predict(image)
    return features.flatten()


Loads VGG16 pretrained on ImageNet, excluding the classification layer
Extract the features

In [ ]:

test_image = image_paths[0]

# HOG Features
hog_features = extract_hog_features(test_image)
print("HOG Features:", hog_features.shape)

# Circle Detection
circles = detect_circles(test_image)
print("Detected Circles:", circles)

# VGG16 Features
vgg16_features = extract_vgg16_features(test_image)
print("VGG16 Features:", vgg16_features.shape)





HOG Features: (8100,)
Detected Circles: None
58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 833ms/step
VGG16 Features: (25088,)


hog_features = long feture vectors representing the texture and the edge pattern
circles = Print None if there is no circle otherwise will print parameters like center co ordinates and radius
Prints the shape

In [ ]:
combined_features = np.concatenate((hog_features, vgg16_features), axis=0)
print("Combined Features:", combined_features.shape)


Combined Features: (33188,)


Combining VGG16 and HOG to a singular vector



---

